In [1]:
import pandas as pd
import folium
from pymongo import MongoClient

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.offices
datos_mongo = db.offices


puntuaciones= {
    'games_video' : 100,
    'software' : 50,
    'BigCompany' : 25,
    'Startup' : 40,
    'USA': -80
    }

In [3]:
# Obtener datos de mongo con el indice 2d geo.
def get_geo_data(datos):
    cursor_geo= datos.find()
    df = pd.DataFrame(cursor_geo)
    df = df.drop('_id', axis=1)
    return df

In [4]:
def sum_points(reg):
    points = 0
    points+= puntuaciones[reg.category_code]
    points+= puntuaciones[reg.type]
    if reg.country_code== 'USA': points+= puntuaciones['USA']
    return points

def get_offices_near(reg):
    points= list()
    lat = float(reg.latitude)
    long = float(reg.longitude)
    df_location = pd.DataFrame(datos_mongo.find({
                                "2Dgeo": {
                                 "$near": {
                                   "$geometry": {
                                      "type": "Point" ,
                                      "coordinates": [ long, lat ]
                                   },
                                   "$maxDistance": 3000, # In meters
                                 }
                               }
                            }))
    points.extend(df_location.apply(sum_points, axis=1))
    return sum(points)

In [5]:
def calculate_points(df):
    df['points'] = df.apply(get_offices_near, axis=1)
    return df

In [9]:
def generate_map(row):
    latitud= float(row.latitude)
    longitud= float(row.longitude)
    nombre= str(row['name'])
    m = folium.Map(
    location=[latitud, longitud],
    tiles='Stamen Toner',
    zoom_start=16)

    tooltip = 'Este es tu sitio!'
    folium.Marker([latitud, longitud], 
                  popup=nombre, 
                  tooltip=tooltip, 
                  icon=folium.Icon(color='green')
                  ).add_to(m)
    m.save('point_map.html')
    return m

In [10]:
#Main
df_geo_mongo = get_geo_data(datos_mongo)
df_points = calculate_points(df_geo_mongo)
df_points= df_points.sort_values('points', ascending=False)
mapa= generate_map(df_points[0:1])

In [11]:
mapa